# Dask and kugupu

The generation of the coupling matrix is the most time consuming calculation done in kugupu.
Luckily as there are no dependencies in calculating different frames, the problem is embarrassingly parallel.
Kugupu is able to use the `dask.distributed` package to calculate many frames in parallel,
as show in this notebook

In [1]:
import kugupu as kgp
import MDAnalysis as mda
from dask import distributed

First we load our simulation data as normal:

In [2]:
u = mda.Universe('./datafiles/C6.data', './datafiles/C6.dcd')

u.add_TopologyAttr('names')
namedict = {
    1.008: 'H',
    12.011: 'C',
    14.007: 'N',
    15.999: 'O',
    32.06: 'S',
}
for m, n in namedict.items():
    u.atoms[u.atoms.masses == m].names = n

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


Now now create a `distributed.Client` to assign the work to.
Here we create a Client running on our local machine, however it is also possible to use a much more powerful Client.

In [3]:
c = distributed.Client()
c

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56424 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:56424/status,
Dashboard: http://127.0.0.1:56424/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56425,Workers: 4
Dashboard: http://127.0.0.1:56424/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56436,Total threads: 2
Dashboard: http://127.0.0.1:56437/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:56428,


2025-04-11 12:20:25,115 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 6 memory: 31 MB fds: 171>>
Traceback (most recent call last):
  File "/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
  File "/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/psutil/__init__.py", line 2148, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory


The generation of results uses the same function,
but we include the `client=` keyword to make the calculation happen in parallel.

In [4]:
res = kgp.coupling_matrix(u, 5.0, 'lumo', degeneracy=1, client=c)

2025-04-11T09:08:10.229751+0100 INFO Processing 5 frames
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.

The results generated this way are identical to as before, and can be saved and used is subsequent analysis as normal.